# Comparison with package ```BNPmix```

In [ ]:
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from google.protobuf.internal.decoder import _DecodeVarint32
import sys
sys.path.insert(0, '..')
from proto.py.algorithm_state_pb2 import AlgorithmState
import arviz as az

# Define paths
CSVIN =  '../resources/csv/in/thesis'
CSVOUT = '../resources/csv/out/thesis'
RECORDIO = '../resources/recordio'
# Utility to read file collector, courtesy of
# github.com/mberaha/utils/blob/master/proto_utils/py/recordio.py
def readManyFromFile(filename, msgType):
    out = []
    with open(filename, "rb") as fp:
        buf = fp.read()
    n = 0
    while n < len(buf):
        msg_len, new_pos = _DecodeVarint32(buf, n)
        n = new_pos
        msg_buf = buf[n:n+msg_len]
        try:
            msg = msgType()
            msg.ParseFromString(msg_buf)
            out.append(msg)
            n += msg_len
        except Exception as e:
            break
    return out

## ```galaxy``` dataset

In [ ]:
# Read data and grid
galaxy_data = np.genfromtxt(f"{CSVIN}/galaxy.csv")
galaxy_grid = np.genfromtxt(f"{CSVIN}/galaxy_grid.csv")

In [ ]:
# Read densities
algo = "Neal2"
mix = "PY"
fdens1 = f"{CSVOUT}/galaxy_dens_{algo}_{mix}.csv"
galaxy_dens_1 = np.genfromtxt(fdens1, delimiter=',')
fdens2 = f"{CSVOUT}/galaxy_dens_bnpmix.csv"
galaxy_dens_2 = np.genfromtxt(fdens2, delimiter=',')

In [ ]:
# Plot densities
fig = plt.figure(figsize=(14,8))
plt.hist(galaxy_data, density=True, color='lightgray')
# Means use 1-in-2 thinning
dens1 = np.exp(np.mean(galaxy_dens_1[0::2], axis=0))
dens2 = np.mean(galaxy_dens_2[0::2], axis=0)
# Plot densities
plt.plot(galaxy_grid, dens1)
plt.plot(galaxy_grid, dens2)
plt.legend(('bayesmix', 'bnpmix'))